In [43]:
import os
import sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, os.path.join(cwd, "../"))

from src.models import additive_attention as attention_module
import torch
import importlib

importlib.reload(attention_module)



<module 'src.models.additive_attention' from '/media/abhinav/Projects/MultiRes/student_life/notebooks/../src/models/additive_attention.py'>

In [44]:
num_features = 9
seq_len = 24
encoder_hidden_size = 128
num_layers = 1
is_cuda = False
dropout_p = 0

# Dummy input seq for testing.
input_seq = torch.ones((seq_len , num_features))
input_seq = input_seq.unsqueeze(0)
print("input_seq shape", input_seq.shape)


####### Model Initialization #######
encoder = attention_module.AttentionEncoder(num_features,
                                   encoder_hidden_size, 
                                   num_layers ,
                                   is_cuda,
                                   dropout_p)

encoder_output_size, context_vector_size = encoder.get_encoder_dimensions()
print("encoder_output_size {}, context_vector_size {}".format(encoder_output_size, context_vector_size))

attention = attention_module.AdditiveAttention(encoder_output_size=encoder_output_size, 
                                               context_vector_size=context_vector_size)

expected_context_vector_from_attention = attention_module.ExpectedContextVectorAfterAttention(attention)

decoder = attention_module.AttentionDecoder(encoder_output_size=encoder_output_size, decoder_hidden_size=context_vector_size,
                           attention=attention,
                           input_size=num_features,
                           output_size=num_features,
                           dropout_p=0)

####### Forward Pass #######

encoder_outputs, context_vector = encoder(input_seq)
print("encoder_outputs {} , context_vector {}".format(encoder_output_size, context_vector_size))

weight_vector = attention(encoder_outputs, context_vector)
print("weight_vector", weight_vector.shape)

expected_encoder_output = expected_context_vector_from_attention(encoder_outputs, context_vector)
print("expected_encoder_output", expected_encoder_output.shape)

input_vector = input_seq[:, -1, :].unsqueeze(0)
print("input_vector", input_vector.shape)

decoder_output, _ = decoder(input_vector=input_vector,
        encoder_outputs=encoder_outputs,
        previous_decoder_hidden_state=context_vector)

print("decoder_output", decoder_output.shape)


input_seq shape torch.Size([1, 24, 9])
encoder_output_size 256, context_vector_size 128
encoder_outputs 256 , context_vector 128
weight_vector torch.Size([1, 24])
expected_encoder_output torch.Size([1, 1, 256])
input_vector torch.Size([1, 1, 9])
rnn_input torch.Size([1, 1, 265])
Only rnn output torch.Size([1, 1, 128])
rnn_output torch.Size([1, 1, 128]) input_vector torch.Size([1, 1, 9]) expected_encoder_output torch.Size([1, 1, 256])
decoder_output torch.Size([1, 1, 9])
